In [2]:
! pip install datasets seqeval

In [3]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Ignore les pairs ou sont set les labels à -100 (inutile pour le calcule des perfs)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


In [5]:
from datasets import load_dataset, load_metric
from transformers import CamembertTokenizerFast, CamembertForTokenClassification
import transformers
from transformers import  TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:

            # Les tokens génerer par le tokenizer son set à -100
            if word_idx is None:
                label_ids.append(-100)
            # Pour chaque premier token d'un mot on lui attribut le label
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # Les subtokens quant à eut sont set également à -100
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

#load le dataset depuis huggingFace
datasets = load_dataset("Babelscape/wikineural")

#les différents labels
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
labels_vocab = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
labels_vocab_reverse = {v:k for k,v in labels_vocab.items()}

#On utilisera seulement les datasets français
train_dataset = datasets["train_fr"]
test_dataset = datasets["test_fr"]
val_dataset = datasets["val_fr"]

first = datasets["train_fr"][0]

tokenizer = CamembertTokenizerFast.from_pretrained("camembert-base")

train_tokenized = train_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized = test_dataset.map(tokenize_and_align_labels, batched=True)
val_tokenized = val_dataset.map(tokenize_and_align_labels, batched=True)

model = CamembertForTokenClassification.from_pretrained("camembert-base", num_labels=len(label_list), label2id=labels_vocab, id2label=labels_vocab_reverse)


In [6]:
import os

num_steps = len(train_dataset) // 16
model_name = "camembert-base"
args = TrainingArguments(
    "Finetuned-camem-ner",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_steps=num_steps,
    save_steps=num_steps,
)

#applique le padding sur no données tokeniser
data_collator = DataCollatorForTokenClassification(tokenizer)
#permet d'évaluer les performances du model, seqeval pouvant évaluer celle d'un modèle de NER
metric = load_metric("seqeval")
#cette metric a besoin de la liste des labels
labels = [label_list[i] for i in first["ner_tags"]]

metric.compute(predictions=[labels], references=[labels])

#definition du Trainer
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

#désactive wandb qui est utilisé par défaut par le trainer
os.environ["WANDB_DISABLED"] = "true"

In [7]:
#phase de fine tuning et d'évaluation
trainer.train()

trainer.evaluate()

In [8]:
#sauvegarde du model
trainer.save_model('models/mycamembert-base-ner')

In [ ]:
#Zip du modèle pour être téléchargé si souhaiter
!zip -r model.zip models/mycamembert-base-ner

In [9]:

import torch

#test sur plusieurs paragraphe de texte notre reconnaissance d'entité nommée

#sentence = "Les Lumières sont un mouvement culturel, philosophique, littéraire et intellectuel qui émerge dans la seconde moitié du XVIIe siècle avec des philosophes comme Descartes, Spinoza, Locke, Bayle et Newton, avant de se développer dans toute l'Europe, notamment en France, au XVIIIe siècle. Par extension, on a donné à cette période le nom de siècle des Lumières. "
sentence = "Facebook est fondé en 2004 par Mark Zuckerberg et ses camarades de l'université Harvard : Chris Hughes, Eduardo Saverin, Andrew McCollum et Dustin Moskovitz. D'abord réservé aux étudiants de cette université, il s'est ensuite ouvert à d'autres universités américaines avant de devenir accessible à tous en septembre 2006."
#sentence = "Charles André Joseph Marie de Gaulle naît le 22 novembre 1890 à 4 heures du matin, au 9 rue Princesse à Lille. Il est baptisé quelques heures après sa naissance en l'église Saint-André de Lille : son parrain est son oncle Gustave de Corbie et sa marraine sa tante Lucie Maillot née Droulers."
tokenized_sentence = tokenizer.encode(sentence)
input_ids = torch.tensor([tokenized_sentence])

with torch.no_grad():
    output = model(input_ids.cuda())
    label_indices = np.argmax(output[0].to('cpu').numpy(),axis=2)
    
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])

new_tokens, new_labels = [], []
#une liste d'execptions nécessaire pour pouvoir bien relier les tokens
execeptions = ['<s>', '.', ',' ':', '?', '!']
for token, label_idx in zip(tokens, label_indices[0]):
    # Si le token commence par le caractère spécial ou est dans les execptions alors le token n'a pas besoin d'etre ajusté
    if token.startswith('▁') or token in execeptions:
        new_labels.append(label_list[label_idx])
        new_tokens.append(token)
    else:
        # Sinon on relie les tokens pour obtenir le mots entier
        new_tokens[-1] = new_tokens[-1] + token

#Retirer le caractère spécial
for x in range(len(new_tokens)):
    new_tokens[x] = new_tokens[x].replace('▁', '')
    
names = []
curr_name = ""

#On parcourt les tokens et récupere les tokens avec les labels correspondant à un noms
for i in range(len(new_tokens)):
    if new_labels[i] != 'O':
        leb = new_labels[i]
        if leb == "B-PER":
            if curr_name != "":
                curr_name += " "
            curr_name += new_tokens[i]
        
        if leb == "I-PER":
            if curr_name != "":
                curr_name += " "
            curr_name += new_tokens[i]

    else:
        if curr_name != "":
            names.append(curr_name)
            curr_name = ""

print(names)
cleaned_names = []
for v in range(len(names)):
   n = names[v].split(", ")
   for m in n:
        cleaned_names.append(m)
print(cleaned_names)

In [10]:
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf

class DistillationTrainingArguments(TrainingArguments):
    def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.alpha = alpha
        self.temperature = temperature

class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher = teacher_model
        self._move_model_to_device(self.teacher,self.model.device)
        self.teacher.eval()
    #override de la méthode compute_loss
    def compute_loss(self, model, inputs, return_outputs = False):
        self._move_model_to_device(model,self.model.device)
        #forward pass du student model
        outputs_stu = model(**inputs)
        # Récupération de la loss
        st_loss = outputs_stu.loss
        
        with torch.no_grad():
            outputs_tea = self.teacher(**inputs)
            
        assert outputs_stu.logits.size() == outputs_tea.logits.size()
        # application de la kl divergence
        loss_function = nn.KLDivLoss(reduction="batchmean")
        loss_logits = (loss_function(
             F.log_softmax(outputs_stu.logits / self.args.temperature, dim=-1),
             F.softmax(outputs_tea.logits / self.args.temperature, dim=-1)) * (self.args.temperature ** 2))

        loss_logits /= (outputs_stu.logits.shape[1] - 1)
        loss = self.args.alpha * loss_logits + (1. - self.args.alpha) * st_loss
        return (loss, outputs_stu) if return_outputs else loss

In [11]:
student_args = DistillationTrainingArguments(
    "distil-ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_steps=num_steps,
    save_steps=num_steps,
    temperature=2.0
)

# chargement du teacher et student model
teacher_checkpoint = "models/mycamembert-base-ner"
student_checkpoint = "cmarkea/distilcamembert-base"
device = torch.device("cuda")
teacher_model = CamembertForTokenClassification.from_pretrained(teacher_checkpoint, num_labels=len(label_list), label2id=labels_vocab, id2label=labels_vocab_reverse).to(device)
student_model = CamembertForTokenClassification.from_pretrained(student_checkpoint, num_labels=len(label_list), label2id=labels_vocab, id2label=labels_vocab_reverse).to(device)

student_tokenizer = CamembertTokenizerFast.from_pretrained(student_checkpoint)
data_collator = DataCollatorForTokenClassification(student_tokenizer)

# Définition du distil trainer
distil_trainer = DistillationTrainer(
    model=student_model,
    teacher_model=teacher_model,
    args=student_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    data_collator=data_collator,
    tokenizer=student_tokenizer,
    compute_metrics=compute_metrics)

In [12]:
distil_trainer.train()

In [13]:
#sauvegarde du modèle distiller
distil_trainer.save_model('models/mydistil-camemebert-ner')

In [ ]:
#pour le téléchargement du modèle distiller
!zip -r modeldistil.zip models/mydistil-camemebert-ner

In [14]:

import torch

#test sur plusieurs paragraphe de texte notre reconnaissance d'entité nommée

#sentence = "Les Lumières sont un mouvement culturel, philosophique, littéraire et intellectuel qui émerge dans la seconde moitié du XVIIe siècle avec des philosophes comme Descartes, Spinoza, Locke, Bayle et Newton, avant de se développer dans toute l'Europe, notamment en France, au XVIIIe siècle. Par extension, on a donné à cette période le nom de siècle des Lumières. "
sentence = "Facebook est fondé en 2004 par Mark Zuckerberg et ses camarades de l'université Harvard : Chris Hughes, Eduardo Saverin, Andrew McCollum et Dustin Moskovitz. D'abord réservé aux étudiants de cette université, il s'est ensuite ouvert à d'autres universités américaines avant de devenir accessible à tous en septembre 2006."
#sentence = "Charles André Joseph Marie de Gaulle naît le 22 novembre 1890 à 4 heures du matin, au 9 rue Princesse à Lille. Il est baptisé quelques heures après sa naissance en l'église Saint-André de Lille : son parrain est son oncle Gustave de Corbie et sa marraine sa tante Lucie Maillot née Droulers."
tokenized_sentence = tokenizer.encode(sentence)
input_ids = torch.tensor([tokenized_sentence])

with torch.no_grad():
    output = student_model(input_ids.to(device))
    label_indices = np.argmax(output[0].to('cpu').numpy(),axis=2)
    
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])

new_tokens, new_labels = [], []
#une liste d'execptions nécessaire pour pouvoir bien relier les tokens
execeptions = ['<s>', '.', ',' ':', '?', '!']
for token, label_idx in zip(tokens, label_indices[0]):
    # Si le token commence par le caractère spécial ou est dans les execptions alors le token n'a pas besoin d'etre ajusté
    if token.startswith('▁') or token in execeptions:
        new_labels.append(label_list[label_idx])
        new_tokens.append(token)
    else:
        # Sinon on relie les tokens pour obtenir le mots entier
        new_tokens[-1] = new_tokens[-1] + token

#Retirer le caractère spécial
for x in range(len(new_tokens)):
    new_tokens[x] = new_tokens[x].replace('▁', '')
    
names = []
curr_name = ""

#On parcourt les tokens et récupere les tokens avec les labels correspondant à un noms
for i in range(len(new_tokens)):
    if new_labels[i] != 'O':
        leb = new_labels[i]
        if leb == "B-PER":
            if curr_name != "":
                curr_name += " "
            curr_name += new_tokens[i]
        
        if leb == "I-PER":
            if curr_name != "":
                curr_name += " "
            curr_name += new_tokens[i]

    else:
        if curr_name != "":
            names.append(curr_name)
            curr_name = ""

print(names)
cleaned_names = []
for v in range(len(names)):
   n = names[v].split(", ")
   for m in n:
        cleaned_names.append(m)
print(cleaned_names)

In [23]:
from transformers import pipeline
#import des pipeline aidant à la classification à partir d'un modèle

In [35]:
# Test la classification du modèle sur le dataset de validation
predictions, labels, _ = distil_trainer.predict(val_tokenized)
predictions = np.argmax(predictions, axis=2)

true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
predictions, labels, _ = trainer.predict(val_tokenized)
predictions = np.argmax(predictions, axis=2)

true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
%%time
#test de la vitesse des 2 modèles pour 1000 phrases de texte.
teacher_model = teacher_model.to('cpu')
for idx in range(1000):
    nlp = pipeline("ner", model=teacher_model, tokenizer=tokenizer)
    joined = ' '.join(val_dataset[idx]['tokens'])
    ner_results = nlp(joined)

In [ ]:
%%time
#test de la vitesse des 2 modèles pour 1000 phrases de texte.)
student_model = student_model.to("cpu")
for idx in range(1000):
    nlp = pipeline("ner", model=student_model, tokenizer=student_tokenizer)
    joined = ' '.join(val_dataset[idx]['tokens'])
    ner_results = nlp(joined)